# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import sys
import json


import nbformat

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import mbrain as mb

# Jupyter

In [4]:
with open('Example.ipynb', 'r') as f:
    nb = nbformat.read(f, as_version=4)

In [5]:
cell = nb['cells'][3]

In [6]:
mb.is_flashcard(nb['cells'][3])

True

In [7]:
print(cell.source)

<!-- {"id": "1560109630656"} -->

**Another question goes here**

Another good and clear answer goes here

* maybe a bullet point
* or make it two
* bullet with $x = 4$ math

$$ x = 2^4 $$

```python
def example_code():
    return 'hohoho'
```

![image.png](attachment:image.png)


In [8]:
meta, head, body, attachments = mb.process_cell(cell)
print('META:\n', meta, '\n')
print('HEAD:\n', head, '\n')
print('BODY:\n', body, '\n')
print('ATTACHMENTS:')
display(attachments)

META:
 {'id': '1560109630656'} 

HEAD:
 Another question goes here 

BODY:
 
<div class="cell border-box-sizing text_cell rendered"><div class="prompt input_prompt">
</div><div class="inner_cell" style="text-align: left">
<div class="text_cell_render border-box-sizing rendered_html">
<p>Another good and clear answer goes here</p>
<ul>
<li>maybe a bullet point</li>
<li>or make it two</li>
<li>bullet with \(x = 4\) math</li>
</ul>
\[ x = 2^4 \]<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">example_code</span><span class="p">():</span>
    <span class="k">return</span> <span class="s1">&#39;hohoho&#39;</span>
</pre></div>
<p><img src="9ea02ea0be30cd2940f46a9d255628ee5bdd6d5b08a169e6728655ccb9902776"></p>

</div>
</div>
</div>
 

 

ATTACHMENTS:


{'image.png': KeyValue(key='9ea02ea0be30cd2940f46a9d255628ee5bdd6d5b08a169e6728655ccb9902776', value='iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAABHNCSVQICAgIfAhkiAAAACNJREFUCFtjZACC/2AEAYwMDIyMyAIwCSYYA5nGKgg0AtNMAIZJBgYMfgDDAAAAAElFTkSuQmCC')}

In [9]:
new_source = mb.put_meta(cell.source, anki_id=meta['id'])
print(new_source)

<!-- {"id": "1560109630656"} -->

**Another question goes here**

Another good and clear answer goes here

* maybe a bullet point
* or make it two
* bullet with $x = 4$ math

$$ x = 2^4 $$

```python
def example_code():
    return 'hohoho'
```

![image.png](attachment:image.png)


In [10]:
cell.source = new_source

In [11]:
with open('Example.ipynb', 'w') as f:
    nbformat.write(nb, f)

# Anki Connect

In [12]:
mb.anki_test_db()

In [13]:
mb.anki_get_decks()

['Taxonomy',
 'Body - Training',
 'Default',
 'Master Deck',
 'zz Input',
 'zz Input 2',
 'zz_Temp',
 'Testing']

In [14]:
mb.anki_find_notes('Testing')

[]

In [15]:
id_ = mb.anki_add_note(deck='Testing', front='Question', back='Answer')
print(id_)

1560108885681


In [16]:
mb.anki_get_note(id_)

('Question', 'Answer')

In [17]:
mb.anki_update_note(id_, front='Question2', back='Answer2')

In [18]:
mb.anki_get_note(id_)

('Question2', 'Answer2')

In [19]:
mb.anki_delete_note(id_)

In [20]:
try:
    mb.anki_get_note(id_)
except KeyError:
    print('Note does not exist')

Note does not exist


**Media Files**

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg==" />

In [34]:
data = 'iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAABHNCSVQICAgIfAhkiAAAACNJ' \
       'REFUCFtjZACC/2AEAYwMDIyMyAIwCSYYA5nGKgg0AtNMAIZJBgYMfgDDAAAAAElFTkSuQmCC'
mb.anki_add_or_replace_media('reddot.png', data=data)

In [35]:
mb.anki_get_media('reddot.png')

'iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAABHNCSVQICAgIfAhkiAAAACNJREFUCFtjZACC/2AEAYwMDIyMyAIwCSYYA5nGKgg0AtNMAIZJBgYMfgDDAAAAAElFTkSuQmCC'

# Jupyter -> Anki

In [24]:
with open('Example.ipynb', 'r') as f:
    nb = nbformat.read(f, as_version=4)

In [25]:
note_ids = mb.anki_find_notes('Testing')
print(note_ids)

['1560110713706', '1560110713781']


In [26]:
commands = []

for cell in nb['cells']:
    if not mb.is_flashcard(cell):
        continue
        
    meta, head, body, attachments = mb.process_cell(cell)
    cmd = mb.figure_out_command(meta, head, body, note_ids)
    if cmd is not None:
        cmd.deck = 'Testing'
        cmd.cell = cell
        cmd.attachments = attachments
        commands.append(cmd)

In [27]:
[c.cmd + ': ' + c.head for c in commands]

[]

In [28]:
for cmd in commands:
    mb.execute_command(cmd)

In [29]:
with open('Example.ipynb', 'w') as f:
    nbformat.write(nb, f)